In [1]:
from arcgis.gis import GIS
import os
import datetime
# Get the current date and time
current_datetime = datetime.datetime.now()
# Format the date and time as YYYYMMDDhhmmss
formatted_datetime = current_datetime.strftime("%Y%m%d%H%M%S")

In [2]:
#login parameters:
output_location = r'C:\temp' #LOCATION WHERE OUTPUTS WILL BE SAVED
USERNAME = '' #YOUR_DIEM_USERNAME
PASS = ''# YOUR_PASSWORD
#survey parameters:
iso3 = ''#ISO3 OF THE COUNTRY - empty string for selecting all countries
round = '' # DIEM SURVEY ROUND NUMBER (integer - empty string for selecting all rounds)

In [3]:
# Initialize the where clause
where_clause = "1=1"  # Default to select all records if both conditions are empty

# Check the conditions and update the where clause accordingly
if iso3 and round:
    where_clause = f"adm0_iso3 = '{iso3}' and round = {round}"
elif iso3:
    where_clause = f"adm0_iso3 = '{iso3}'"
elif round:
    where_clause = f"round = {round}"

In [4]:
gis = GIS('https://hqfao-hub.maps.arcgis.com/home/group.html?id=4eca32aadf664a8f9a59d2dd68b7444d&view=list#content', USERNAME, PASS)
# Get the group using the Group ID (From the group URL)
group = gis.groups.get("4eca32aadf664a8f9a59d2dd68b7444d")
group_items = group.content()
for group_item in group_items:
    print(f"Processing the {group_item.title} dataset.")
    subset_feature_layer = group_item.layers[0]
    df = subset_feature_layer.query(where=where_clause, out_fields = "*", returnGeometry=False, as_df=True)
    df.to_csv(os.path.join(output_location,group_item.title + "_" + formatted_datetime + ".csv"))
    print(f"Saved the {group_item.title} data in {output_location} .\n")
print("Process completed")

Processing the DIEM aggregated data (crop production thematic area) dataset.
Saved the DIEM aggregated data (crop production thematic area) data in C:\temp .

Processing the DIEM aggregated data (food security thematic area) dataset.
Saved the DIEM aggregated data (food security thematic area) data in C:\temp .

Processing the DIEM aggregated data (income, shocks and needs thematic areas) dataset.
Saved the DIEM aggregated data (income, shocks and needs thematic areas) data in C:\temp .

Processing the DIEM aggregated data (livestock production thematic area) dataset.
Saved the DIEM aggregated data (livestock production thematic area) data in C:\temp .

Process completed
